### Import

In [22]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
from python_module.pricing_model import BSMModel
from python_module.pricing_model import HestonHullWhiteModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

### Sensitivity

In [23]:
S           = 100
K           = 100
T_min       = 1 
T_max       = 250
sigma       = 0.2
r           = 0
option_type = 'call'

S_list = np.linspace(80, 120, 100)
T_list = [T_min/250, T_max/250]
results = dict()
for S in tqdm(S_list):
    for T in T_list:
        pricing_results = BSMModel.compute_option_with_spot(S=S, K=K, T=T, r=r, g=0, q=0, sigma=sigma, option_type=option_type, compute_greeks=True)
        results[(S, T)] = pricing_results
results_df = pd.DataFrame(results).transpose()
results_df.index.names = ['S', 'T']
results_df = results_df.reset_index()
for field in ['price', 'delta', 'vega', 'theta', 'vanna', 'volga']:
    for T in [T_max, T_min]:
        t = T / 250
        df_temp = results_df[results_df['T'] == t]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_temp['S'], y=df_temp[field], mode='lines', name=f'T={T:.0f}'))
        fig.update_layout(title=f'{field.capitalize()} vs S for T={T:.2f}', xaxis_title='S', yaxis_title=field.capitalize())
        fig.show()

100%|██████████| 100/100 [00:00<00:00, 2132.25it/s]


### HestonHullWhiteModel

In [24]:
# -------
# Price vanilla option without stochastic rates
# -------

# Define input parameters for Heston Hull White model
S0 = 100.0        # Initial stock price
v0 = 0.2**2       # Initial variance
r0 = 0.00         # Initial short rate
b0 = 0.0          # Initial state variable for bonds

# Variance process parameters
kappa_v = 1.0     # Mean reversion speed for variance
theta_v = 0.2**2  # Long-term variance
sigma_v = 0.2     # Volatility of variance

# Interest rate process parameters
kappa_r = 10.0     # Mean reversion speed for rates
theta_r = 0.00    # Long-term interest rate
sigma_r = 0.00001  # Volatility of interest rate

# Correlation parameters
rho_sv = -0.5     # Stock-variance correlation
rho_sr = 0.9      # Stock-rate correlation
rho_vr = -0.5     # Variance-rate correlation

# Simulation parameters
T = 1.0           # Time horizon in years
M = 252           # Number of simulations
N = 10000         # Number of time steps
seed = True       # Use seed for reproducibility
seed_value = 44   # Seed value

S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)
call_price = (pd.DataFrame(S).iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('call price, ', call_price)

call price,  7.927970425907604


In [25]:
# -------
# Add stochastic rates - > decrease eq vol to line up with previous price
# -------

v0 = 0.16**2       # Initial variance
theta_v = 0.16**2  # Long-term variance
sigma_r = 0.5  # Volatility of interest rate
S, v, r = HestonHullWhiteModel.compute_montecarlo(
    S0=S0, v0=v0, r0=r0, b0=b0,
    kappa_v=kappa_v, theta_v=theta_v, sigma_v=sigma_v,
    kappa_r=kappa_r, theta_r=theta_r, sigma_r=sigma_r,
    rho_sv=rho_sv, rho_sr=rho_sr, rho_vr=rho_vr,
    T=T, N=N, M=M,
    seed=seed, seed_value=seed_value)
call_price = (pd.DataFrame(S).iloc[-1].add(-100).clip(lower=0).multiply(np.exp(-pd.DataFrame(r).multiply((T / M)).sum()))).mean()
print('call price, ', call_price)

call price,  7.917979234888703


### Numerical Derivative and P&L attribution

In [26]:
def compute_numeric_derivative(pricing_fun, base_params, param_names, h=None):
    """
    Numerically compute derivative(s) of pricing_fun at base_params.

    - pricing_fun: callable(**kwargs) -> scalar price
    - base_params: dict of parameters to pass to pricing_fun
    - param_names: str or (str, str)
        * If str -> first derivative w.r.t that parameter (central difference)
        * If (p, p) -> second derivative w.r.t p (second central difference)
        * If (p, q) with p != q -> mixed second partial d2/dp dq (central)
    - h: optional bump size (scalar or dict mapping param->h). If None, automatic h used.

    Returns numeric derivative (float).
    """
    import numpy as np

    def safe_eval(params):
        val = pricing_fun(**params)
        a = np.asarray(val)
        if a.size != 1:
            raise ValueError("pricing_fun must return a scalar-like value")
        return float(a.item())

    eps = np.finfo(float).eps
    # normalize param_names to tuple
    if isinstance(param_names, str):
        params = (param_names,)
    else:
        params = tuple(param_names)

    # helper to compute adaptive h for a single param
    def get_h_for(name, x):
        if h is None:
            return (eps ** (1/3)) * (abs(x) + 1.0)
        if isinstance(h, dict):
            return float(h.get(name, (eps ** (1/3)) * (abs(x) + 1.0)))
        return float(h)

    # single derivative (first)
    if len(params) == 1:
        p = params[0]
        x0 = float(base_params[p])
        hh = get_h_for(p, x0)
        p_plus = dict(base_params); p_plus[p] = x0 + hh
        p_minus = dict(base_params); p_minus[p] = x0 - hh
        f_plus = safe_eval(p_plus)
        f_minus = safe_eval(p_minus)
        return (f_plus - f_minus) / (2.0 * hh)

    # two params -> second derivative or mixed partial
    if len(params) == 2:
        p, q = params
        x_p = float(base_params[p])
        x_q = float(base_params[q])
        h_p = get_h_for(p, x_p)
        h_q = get_h_for(q, x_q)

        if p == q:
            # second derivative w.r.t same parameter
            p_plus = dict(base_params); p_plus[p] = x_p + h_p
            p_minus = dict(base_params); p_minus[p] = x_p - h_p
            f_plus = safe_eval(p_plus)
            f0 = safe_eval(base_params)
            f_minus = safe_eval(p_minus)
            return (f_plus - 2.0 * f0 + f_minus) / (h_p ** 2)
        else:
            # mixed partial: central 4-point formula
            pp = dict(base_params); pp[p] = x_p + h_p; pp[q] = x_q + h_q
            pm = dict(base_params); pm[p] = x_p + h_p; pm[q] = x_q - h_q
            mp = dict(base_params); mp[p] = x_p - h_p; mp[q] = x_q + h_q
            mm = dict(base_params); mm[p] = x_p - h_p; mm[q] = x_q - h_q
            f_pp = safe_eval(pp)
            f_pm = safe_eval(pm)
            f_mp = safe_eval(mp)
            f_mm = safe_eval(mm)
            return (f_pp - f_pm - f_mp + f_mm) / (4.0 * h_p * h_q)

    raise ValueError("param_names must be a string or a tuple/list of two strings")

In [27]:
# wrapper if your model returns a dict or complex object
def price_scalar(**kwargs):
    return BSMModel.compute_option_with_forward(**kwargs)  # must return scalar

base = dict(F=100, K=100, T=1, r=0, sigma=0.3, option_type='call', compute_greeks=False)

delta = compute_numeric_derivative(price_scalar, base, 'F')
gamma = compute_numeric_derivative(price_scalar, base, ('F','F'))
vega = compute_numeric_derivative(price_scalar, base, ('sigma'))
theta = compute_numeric_derivative(price_scalar, base, ('T'))
vanna = compute_numeric_derivative(price_scalar, base, ('F','sigma'))
volga = compute_numeric_derivative(price_scalar, base, ('sigma','sigma'))

In [28]:
delta, gamma, vega/100, theta/-250, vanna, volga

(0.5596176923562872,
 0.013149315164148868,
 0.39447933090164694,
 -0.02366875985292543,
 0.1972393877324846,
 -2.958673546784521)

In [29]:
base = dict(F=100, K=100, T=1, r=0, sigma=0.3, option_type='call', compute_greeks=True)
BSMModel.compute_option_with_forward(**base)

{'price': 11.923538474048499,
 'delta': 0.5596176923702425,
 'gamma': 0.013149311030262964,
 'vega': 0.39447933090788895,
 'theta': -0.023480912554041007,
 'vanna': 0.19723966545394447,
 'volga': -2.958594981809167}

In [34]:
BSMModel.compute_pnl_attribution(F_start=100, F_end=100, sigma_start=0.30, sigma_end=0.30, T_start=252/252, T_end=252/252, K=100, r=0, option_type='call')

{'total_pnl': 0.0,
 'delta_pnl': 0.0,
 'gamma_pnl': 0.0,
 'theta_pnl': 0.0,
 'vega_pnl': 0.0,
 'vanna_pnl': 0.0,
 'volga_pnl': -0.0,
 'unexplained_pnl': 0.0}

### Numerical computation of E[Greek]

In [31]:
S = 100
K = 100
T = 1
r = 0.0
g = 0.0
q = 0.0
sigma = 0.2
n_steps = 252
n_paths = 1000
option_type = 'call'
price_df = BSMModel.compute_montecarlo(S=S, T=T, r=r, g=g, q=q, sigma=sigma, n_steps=n_steps, n_paths=n_paths)
t_df = price_df.copy()
t_df.loc[:, :] = T / n_steps
t_df = t_df.sort_index(ascending=False).cumsum().sort_index()
t_df = t_df - (T / n_steps)
# Compute average slide
bump = -0.3
greek = price_df.copy()
greek.loc[:, :] = np.nan

for index in tqdm(price_df.index):
    for column in price_df.columns:
        
        S = price_df.loc[index, column]
        T = t_df.loc[index, column]
        
        result = BSMModel.compute_option_with_spot(S=S, K=K, T=T, r=r, g=g, q=q, sigma=sigma, option_type=option_type, compute_greeks=True)
        delta_pnl = result['delta'] * S * 0.3
        
        pv_bumped = BSMModel.compute_option_with_spot(S=S*(1+bump), K=K, T=T, r=r, g=g, q=q, sigma=sigma, option_type=option_type, compute_greeks=False)
        option_pnl = pv_bumped - result['price']
        
        greek.loc[index, column] = option_pnl + delta_pnl
        
print(greek.iloc[:-1].mean().mean())
print(greek.iloc[:-1].mean(axis=1).mean())

100%|██████████| 253/253 [01:07<00:00,  3.75it/s]

8.513813160920856
8.513813160920856
